In [1]:
import pandas as pd
import re
%config InlineBackend.figure_format = 'retina'

In [2]:
# Read from all experiments
csv_files = [
    "crypto_asymmetric_20220323_1659_sections.csv",
    "crypto_hashing_20220323_1725_sections.csv",
]

df = pd.concat([
    pd.read_csv(f"../measurements/{fn}", usecols=["start", "end", "name", "energy", "power"]) for fn in csv_files
], ignore_index=True)
df["duration"] = df.end - df.start

print(f"Read {len(df)} rows")

Read 466 rows


In [3]:
# Compute normalized power/energy
idle_sections = ["idle-offline-begin"]
idle_mean_power = df[df.name.isin(idle_sections)].power.mean()
df["power_normalized"] = df.power - idle_mean_power
df["energy_normalized"] = df.power_normalized * df.duration
print(f"mean({idle_sections}): {idle_mean_power:.1f}mW")

mean(['idle-offline-begin']): 28.6mW


In [4]:
def get_section_names(df, include_regex=".*", exclude_regex=None):
    include_regex = re.compile(include_regex)
    exclude_regex = re.compile(exclude_regex) if exclude_regex else None
    return [
        x
        for x in df.name.unique()
        if include_regex.search(x) and not (exclude_regex.search(x) if exclude_regex else False)
    ]

In [5]:
def humanify(name):
    try:
        return {
            "keygen-rsa1024": "Gen RSA-1024",
            "keygen-rsa2048": "Gen RSA-2048",
            "keygen-rsa4096": "Gen RSA-4096",
            "sign-rsa1024": "Sign RSA-1024",
            "sign-rsa2048": "Sign RSA-2048",
            "sign-rsa4096": "Sign RSA-4096",
            "verify-rsa1024": "Verify RSA-1024",
            "verify-rsa2048": "Verify RSA-2048",
            "verify-rsa4096": "Verify RSA-4096",
            
            "keygen-ecdsa224": "Gen EC-224",
            "keygen-ecdsa256": "Gen EC-256",
            "keygen-ecdsa384": "Gen EC-384",
            "keygen-ecdsa521": "Gen EC-521",
            "sign-ecdsa224": "Sign EC-224",
            "sign-ecdsa256": "Sign EC-256",
            "sign-ecdsa384": "Sign EC-384",
            "sign-ecdsa521": "Sign EC-521",
            "verify-ecdsa224": "Verify EC-224",
            "verify-ecdsa256": "Verify EC-256",
            "verify-ecdsa384": "Verify EC-384",
            "verify-ecdsa521": "Verify EC-521",
            
            "sphinx-single": "Sphinx (1x)",
            "sphinx-multi-10": "Sphinx (10x)",
            "sphinx-multi-100": "Sphinx (100x)",
            
            "hash-sha256": "SHA256",
            "hash-sha512": "SHA512",
            
            "argon2-t1m16p1-i": "Argon2 (1x 16MiB, I)",
            "argon2-t1m16p1-d": "Argon2 (1x 16MiB, D)",
            "argon2-t1m16p1-id": "Argon2 (1x 16MiB)",
            
            "argon2-t2m32p1-id": "Argon2 (2x 32MiB)",
            "argon2-t4m32p1-id": "Argon2 (4x 32MiB)",
            "argon2-t2m64p1-id": "Argon2 (2x 64MiB)",
            
            "argon2-t2m32p2-id": "Argon2 (2x 32MiB p=2)",
            "argon2-t2m32p4-id": "Argon2 (2x 32MiB p=4)",
            
            "argon2-owasp-1": "Argon2 (OWASP-1)",
            "argon2-owasp-2": "Argon2 (OWASP-2)",
            
            "pbkdf2-light-i": "PBKDF2 (light)",
            "pbkdf2-heavy-i": "PBKDF2 (strong)",
            "pbkdf2-owasp": "PBKDF2 (OWASP)",
        }[name]
    except KeyError:
        return name

In [6]:
names_rsa = get_section_names(df, "rsa")
names_ec = get_section_names(df, "ecdsa")
names_sphinx = get_section_names(df, "sphinx")
names_hash = get_section_names(df, "hash")
names_pwhash = get_section_names(df, "argon|pbkdf")

In [7]:
def print_table(d, names):
    for name in names:
        p = d[d.name==name].energy_normalized.mean()
        std = d[d.name==name].energy_normalized.std()
        asterisk = "*" if p < 0.25 else ""
        print(f"{humanify(name)} & {p:.2f}{asterisk} & {std:.2f} \\\\ ")

In [8]:
print("\\midrule")
print_table(df, names_rsa)
print("\\midrule")
print_table(df, names_ec)
print("\\midrule")
print_table(df, names_sphinx)

\midrule
Gen RSA-1024 & 116.47 & 76.36 \\ 
Gen RSA-2048 & 796.05 & 597.84 \\ 
Gen RSA-4096 & 2898.43 & 2042.77 \\ 
Sign RSA-1024 & 1.88 & 0.35 \\ 
Sign RSA-2048 & 6.22 & 1.22 \\ 
Sign RSA-4096 & 26.73 & 4.68 \\ 
Verify RSA-1024 & 0.34 & 0.10 \\ 
Verify RSA-2048 & 0.50 & 0.08 \\ 
Verify RSA-4096 & 0.75 & 0.10 \\ 
\midrule
Gen EC-224 & 1.11 & 0.06 \\ 
Gen EC-256 & 0.51 & 0.05 \\ 
Gen EC-384 & 2.65 & 0.07 \\ 
Gen EC-521 & 5.25 & 0.27 \\ 
Sign EC-224 & 1.43 & 0.15 \\ 
Sign EC-256 & 0.83 & 0.18 \\ 
Sign EC-384 & 3.24 & 0.16 \\ 
Sign EC-521 & 6.27 & 0.10 \\ 
Verify EC-224 & 1.52 & 0.05 \\ 
Verify EC-256 & 1.54 & 0.08 \\ 
Verify EC-384 & 3.43 & 0.07 \\ 
Verify EC-521 & 6.74 & 0.15 \\ 
\midrule
Sphinx (1x) & 9.66 & 0.31 \\ 
Sphinx (10x) & 86.34 & 8.07 \\ 
Sphinx (100x) & 842.44 & 11.40 \\ 


In [9]:
print("\\midrule")
print_table(df, names_hash)
print("\\midrule")
print_table(df, names_pwhash)

\midrule
SHA256 & 0.26 & 0.23 \\ 
SHA512 & 0.27 & 0.07 \\ 
\midrule
Argon2 (1x 16MiB, I) & 51.33 & 1.62 \\ 
Argon2 (1x 16MiB, D) & 50.07 & 1.91 \\ 
Argon2 (1x 16MiB) & 49.99 & 0.76 \\ 
Argon2 (2x 32MiB) & 221.87 & 6.45 \\ 
Argon2 (4x 32MiB) & 228.22 & 10.38 \\ 
Argon2 (2x 64MiB) & 446.03 & 11.80 \\ 
Argon2 (2x 32MiB p=2) & 176.05 & 0.78 \\ 
Argon2 (2x 32MiB p=4) & 148.03 & 6.02 \\ 
Argon2 (OWASP-1) & 124.27 & 6.59 \\ 
Argon2 (OWASP-2) & 101.68 & 4.19 \\ 
PBKDF2 (light) & 908.39 & 21.34 \\ 
PBKDF2 (strong) & 8964.32 & 50.79 \\ 
PBKDF2 (OWASP) & 6386.29 & 118.00 \\ 
